In [1]:
import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker

In [3]:
!pip install xgboost

    100% |████████████████████████████████| 109.8MB 463kB/s eta 0:00:01
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import data_manipulation as dm
import xgboost as xgb

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [ ]:
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()

region = sess.boto_region.region_name
bucket_name = 'lvp-ion-switching'

train_name = 'train_clean.csv'
test_name = 'test_clean.csv'

train_path = f's3
